# スクレイピングテスト(気象庁)

* 最低気温がない場合があるので、np.nanで置き換える。
* 最高気温、最低気温の取得は比較的容易であるが、地域・都市が複数ある場合は、都市名で検索し特定の値を取得する必要がある。
* 気象庁に問い合わせたところ、商用利用は可能であるが機械的に収集する方法は推奨していない。  
ただし、1日1回とかそういうレベルであれば問題はなさそうである。
* 更新頻度は、5時、11時、17時の3回は確定であるが、情報が変わり次第随時更新する。
* 気象庁＞防災情報＞天気予報のページは301.html～356.htmlまで

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime as dt

# 気象庁の予報ページ（例：大阪府）
scrapy_target_url = 'http://www.jma.go.jp/jp/yoho/331.html'
target_locate = '大阪'
# scrapy_target_url = 'http://www.jma.go.jp/jp/yoho/331.html'
# target_locate = '大阪'
# scrapy_target_url = 'http://www.jma.go.jp/jp/yoho/332.html'
# target_locate = '神戸'

def get_target_htmlbsdata(targeturl):
    '''
    Use BeautifulSoup to return an HTML object
    BeautifulSoupを使用して、HTMLを取得したオブジェクトを返す

    '''

    # BeautifulSoupを使用して、HTMLを取得したオブジェクトを返す
    html = urllib.request.urlopen(targeturl)
    html_data = BeautifulSoup(html, "html.parser")
    return html_data


def rm_escapestring_for_string(target_string):
    # 改行コードとタブを取り除く
    target_string = re.sub(r'\n','',target_string)
    target_string = re.sub(r'\t','',target_string)
    return target_string


def select_records_by_locate(df,locate):
    # 必要なLocateのみのデータフレームにする。
    df = df[df['Locate'] == locate]
    return df

In [2]:
# スクレイピング
bs_htmldata = get_target_htmlbsdata(scrapy_target_url)
maxdata = bs_htmldata.find_all("td",class_="max")
mindata = bs_htmldata.find_all("td",class_="min")
date_now = dt.datetime.now()

In [5]:
# 初期化
df = pd.DataFrame(columns=['最高気温','最低気温'])
ser_max = np.array([])
ser_min = np.array([])
ser_locate = np.array([])

#最高気温取得
for target_item_max in maxdata:
    try:
        ser_max = np.append(ser_max,float(rm_escapestring_for_string(target_item_max.string).replace('度','')))

    except:
        print('最高気温欠損')
        ser_max = np.append(ser_max,np.nan)

# 最低気温取得
for target_item_min in mindata:
    try:
        ser_min = np.append(ser_min,float(rm_escapestring_for_string(target_item_min.string).replace('度','')))
    except:
        print('最低気温欠損')
        ser_min = np.append(ser_min,np.nan)

# 各気温情報の都市名を取得
for item in bs_htmldata.find_all("td",class_="city"):
    ser_locate = np.append(ser_locate,str(item.string))        

# データフレームに格納
df['最高気温'] = ser_max
df['最低気温'] = ser_min
df['Locate'] = ser_locate

# 都市名によるレコードの選定
df = select_records_by_locate(df,target_locate)

# 各気温情報に日付indexを付与
df.index = [date_now.strftime('%Y/%m/%d'),(date_now+pd.offsets.Day(1)).strftime('%Y/%m/%d')]

最低気温：NoData


In [6]:
df

,最高気温,最低気温,Locate
2018/08/27,35.0,NaN,大阪
2018/08/28,34.0,26.0,大阪
